In [ ]:
!pip install chart_studio

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.4/64.4 kB 2.9 MB/s eta 0:00:00


In [106]:
from   sklearn.linear_model    import Ridge
from   sklearn.linear_model    import RidgeCV
from   sklearn.linear_model    import LinearRegression
from sklearn.linear_model import Lasso
from sklearn.linear_model import LassoCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import SGDRegressor
from sklearn.utils import resample
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import AdaBoostRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import RandomizedSearchCV
from scipy.stats import uniform
from   sklearn.metrics         import mean_squared_error, r2_score
import scipy.stats             as     stats
import pandas                  as     pd
import numpy                   as     np
import seaborn                 as     sns
import matplotlib.pyplot       as     plt

import warnings


sns.set(color_codes=True)
warnings.filterwarnings('ignore')

In [107]:
scaler = StandardScaler()

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Dummy Encoding

In [108]:
medals = pd.read_pickle('medals.pkl')
print(medals.shape)
medals.head()

(1166, 16)


,Year,Host,NOC,Athletes,Females,Sports,Events,Medals,Golds,Silvers,Bronzes,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Athletes per Event
0,1900,0,AUS,2,0,2,6,6,3,0,3,5,2,0,3,0.333
1,1900,0,AUT,16,0,4,17,6,0,3,3,6,0,3,3,0.941
2,1900,0,BEL,64,0,10,31,18,6,7,5,17,6,7,4,2.065
3,1900,0,BOH,7,1,4,9,3,0,1,2,2,0,1,1,0.778
4,1900,0,CAN,4,0,1,5,2,1,0,1,2,1,0,1,0.800


In [109]:
medals_expand=medals.copy()
medals_expand.head()
medals_expand.columns

Index(['Year', 'Host', 'NOC', 'Athletes', 'Females', 'Sports', 'Events',
       'Medals', 'Golds', 'Silvers', 'Bronzes', 'Real_Medals', 'Real_Gold',
       'Real_Silver', 'Real_Bronze', 'Athletes per Event'],
      dtype='object')

In [110]:
medals_expand=medals.copy()
def add_lag_features(df, lag=4):
    # 使用groupby对NOC（国家）进行分组
    for i in range(1, lag + 1):
        # 创建前i届奥运会的特征：奖牌数、总数等
        df[f'Gold_Lag{i}'] = df.groupby('NOC')['Real_Gold'].shift(i)
        df[f'Silver_Lag{i}'] = df.groupby('NOC')['Real_Silver'].shift(i)
        df[f'Bronze_Lag{i}'] = df.groupby('NOC')['Real_Bronze'].shift(i)
        df[f'Athletes_Lag{i}'] = df.groupby('NOC')['Athletes'].shift(i)
        df[f'Females_Lag{i}'] = df.groupby('NOC')['Females'].shift(i)
        df[f'Sports_Lag{i}'] = df.groupby('NOC')['Sports'].shift(i)
        df[f'Events_Lag{i}'] = df.groupby('NOC')['Events'].shift(i)
        df[f'Athletes per Event_Lag{i}'] = df.groupby('NOC')['Athletes per Event'].shift(i)
    return(df)
medals_expand=add_lag_features(medals_expand, lag=9)
medals_expand.tail(20)

,Year,Host,NOC,Athletes,Females,Sports,Events,Medals,Golds,Silvers,...,Events_Lag8,Athletes per Event_Lag8,Gold_Lag9,Silver_Lag9,Bronze_Lag9,Athletes_Lag9,Females_Lag9,Sports_Lag9,Events_Lag9,Athletes per Event_Lag9
1146,2024,0,PRK,14,10,7,16,6,0,2,...,23.0,1.609,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1147,2024,0,PUR,51,24,13,30,2,0,0,...,8.0,1.125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1148,2024,0,QAT,13,1,5,10,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1149,2024,0,ROU,96,47,18,57,9,3,4,...,59.0,1.153,20.0,16.0,17.0,124.0,53.0,13.0,86.0,1.442
1150,2024,0,RSA,141,63,24,66,6,1,3,...,50.0,1.000,2.0,4.0,4.0,64.0,4.0,13.0,59.0,1.085
1151,2024,0,SGP,23,16,11,24,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1152,2024,0,SRB,114,44,16,47,5,3,1,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1153,2024,0,SUI,137,66,22,94,7,1,2,...,83.0,1.193,0.0,4.0,4.0,129.0,27.0,18.0,91.0,1.418
1154,2024,0,SVK,28,14,13,31,1,0,0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1155,2024,0,SWE,125,66,21,81,11,4,4,...,113.0,1.611,2.0,11.0,6.0,172.0,42.0,19.0,138.0,1.246


In [111]:
medals_expand=medals_expand.drop(['Medals','Silvers','Bronzes','Golds','Athletes', 'Females', 'Sports', 'Events','Athletes per Event'],axis=1)
medals_expand.head()

,Year,Host,NOC,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Gold_Lag1,Silver_Lag1,Bronze_Lag1,...,Events_Lag8,Athletes per Event_Lag8,Gold_Lag9,Silver_Lag9,Bronze_Lag9,Athletes_Lag9,Females_Lag9,Sports_Lag9,Events_Lag9,Athletes per Event_Lag9
0,1900,0,AUS,5,2,0,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1900,0,AUT,6,0,3,3,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1900,0,BEL,17,6,7,4,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1900,0,BOH,2,0,1,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1900,0,CAN,2,1,0,1,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [112]:
medals_expand.fillna(0,inplace=True)
medals_expand.head()

,Year,Host,NOC,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Gold_Lag1,Silver_Lag1,Bronze_Lag1,...,Events_Lag8,Athletes per Event_Lag8,Gold_Lag9,Silver_Lag9,Bronze_Lag9,Athletes_Lag9,Females_Lag9,Sports_Lag9,Events_Lag9,Athletes per Event_Lag9
0,1900,0,AUS,5,2,0,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,1900,0,AUT,6,0,3,3,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,1900,0,BEL,17,6,7,4,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,1900,0,BOH,2,0,1,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,1900,0,CAN,2,1,0,1,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [113]:
# create dummy variables
model        = pd.get_dummies(data=medals_expand, columns=['NOC'])
model['NOC'] = medals['NOC']
print(model.shape)
model.head()

(1166, 206)


,Year,Host,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Gold_Lag1,Silver_Lag1,Bronze_Lag1,Athletes_Lag1,...,NOC_UKR,NOC_URS,NOC_USA,NOC_UZB,NOC_VEN,NOC_VIE,NOC_YUG,NOC_ZAM,NOC_ZIM,NOC
0,1900,0,5,2,0,3,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,AUS
1,1900,0,6,0,3,3,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,AUT
2,1900,0,17,6,7,4,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,BEL
3,1900,0,2,0,1,1,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,BOH
4,1900,0,2,1,0,1,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,CAN


# Train / Test Split

In [114]:

model = model.reset_index().drop(['index'], axis=1)
print(model.shape)
model.head()

(1166, 206)


,Year,Host,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Gold_Lag1,Silver_Lag1,Bronze_Lag1,Athletes_Lag1,...,NOC_UKR,NOC_URS,NOC_USA,NOC_UZB,NOC_VEN,NOC_VIE,NOC_YUG,NOC_ZAM,NOC_ZIM,NOC
0,1900,0,5,2,0,3,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,AUS
1,1900,0,6,0,3,3,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,AUT
2,1900,0,17,6,7,4,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,BEL
3,1900,0,2,0,1,1,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,BOH
4,1900,0,2,1,0,1,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,CAN


In [115]:
y = model[['Real_Medals','Real_Gold','Real_Silver','Real_Bronze','Year','NOC']]
print(y.shape)
y.head()

(1166, 6)


,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Year,NOC
0,5,2,0,3,1900,AUS
1,6,0,3,3,1900,AUT
2,17,6,7,4,1900,BEL
3,2,0,1,1,1900,BOH
4,2,1,0,1,1900,CAN


In [116]:
X = model.drop(['Real_Medals','Real_Gold','Real_Silver','Real_Bronze'],axis=1)
print(X.shape)
X.head()

(1166, 202)


,Year,Host,Gold_Lag1,Silver_Lag1,Bronze_Lag1,Athletes_Lag1,Females_Lag1,Sports_Lag1,Events_Lag1,Athletes per Event_Lag1,...,NOC_UKR,NOC_URS,NOC_USA,NOC_UZB,NOC_VEN,NOC_VIE,NOC_YUG,NOC_ZAM,NOC_ZIM,NOC
0,1900,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,AUS
1,1900,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,AUT
2,1900,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,BEL
3,1900,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,BOH
4,1900,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,False,False,False,False,False,False,False,False,False,CAN


In [117]:
X_train = X[X['Year'] <  2024]
X_test  = X[X['Year'] == 2024]
X_train = X_train.reset_index().drop(['index'], axis=1)
X_test  = X_test.reset_index().drop(['index'], axis=1)
print(X_train.shape)
print(X_test.shape)

y_train = y[y['Year'] <  2024]
y_test  = y[y['Year'] == 2024]
y_test  = y_test.reset_index().drop(['index'], axis=1)
print(y_train.shape)
print(y_test.shape)

(1084, 202)
(82, 202)
(1084, 6)
(82, 6)


In [50]:

X_Scaled_train = scaler.fit_transform(X_train.drop('NOC',axis=1))
X_Scaled_test = scaler.transform(X_test.drop('NOC',axis=1))


# Linear Regression Training Model

In [51]:
# Create regression objects
#lasso = LassoCV(cv=5, random_state=42)
#regr_golds   = lasso
#regr_silvers = lasso
#regr_bronzes = lasso
# elastic_net_cv = ElasticNetCV(l1_ratio=[0.1, 0.5, 0.7, 0.9, 1], cv=5)
# regr_golds   = elastic_net_cv
# regr_silvers = elastic_net_cv
# regr_bronzes = elastic_net_cv
# svr = SVR(
#     C=1.0,               # 惩罚参数
#     kernel='rbf',        # 核函数类型
#     degree=3,            # 核函数的多项式阶数（如果使用多项式核）
#     gamma='scale',       # 核函数的系数
#     epsilon=0.1,         # 容忍度
#     shrinking=True,      # 启用收缩启发式
#     tol=1e-3             # 收敛容忍度
# )
model = RidgeCV(alphas=np.logspace(-6, 6, 13), store_cv_values=True)
regr_golds   = RidgeCV(alphas=np.logspace(-6, 6, 13), store_cv_values=True)
regr_silvers = RidgeCV(alphas=np.logspace(-6, 6, 13), store_cv_values=True)
regr_bronzes = RidgeCV(alphas=np.logspace(-6, 6, 13), store_cv_values=True)


In [52]:
# Train the models using the training sets
regr_golds.fit(  X_Scaled_train, y_train['Real_Gold'])
regr_silvers.fit(X_Scaled_train, y_train['Real_Silver'])
regr_bronzes.fit(X_Scaled_train, y_train['Real_Bronze'])

RidgeCV(alphas=array([1.e-06, 1.e-05, 1.e-04, 1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01,
       1.e+02, 1.e+03, 1.e+04, 1.e+05, 1.e+06]),
        store_cv_values=True)

# Bootstrapping

In [53]:
def bootstrap_resample(X_predict, y_train):
  # Bootstrap 参数
  n_iterations = 1000
  n_size = len(X_Scaled_train)
  # 存储预测结果
  predictions = np.zeros((n_iterations, len(X_predict)))

  # Bootstrap 采样
  for i in range(n_iterations):
      # 随机采样（有放回）
      X_resample, y_resample = resample(X_Scaled_train, y_train, n_samples=n_size)

      # 训练模型
      model.fit(X_resample, y_resample)

      # 预测
      predictions[i, :] = model.predict(X_predict)

  # 计算预测区间
  lower_bound = np.percentile(predictions, 2.5, axis=0)
  upper_bound = np.percentile(predictions, 97.5, axis=0)
  return lower_bound, upper_bound


In [54]:
# Make predictions using the training sets
y_train['Golds Prediction']   = pd.DataFrame(
    regr_golds.predict(X_Scaled_train), columns=['Golds Prediction'])
y_train['Golds Prediction']   = y_train['Golds Prediction']
y_train['Golds Prediction']   = y_train['Golds Prediction'].clip(lower=0)

y_train['Silvers Prediction'] = pd.DataFrame(
    regr_silvers.predict(X_Scaled_train), columns=['Silvers Prediction'])
y_train['Silvers Prediction'] = y_train['Silvers Prediction']
y_train['Silvers Prediction'] = y_train['Silvers Prediction'].clip(lower=0)

y_train['Bronzes Prediction'] = pd.DataFrame(
    regr_bronzes.predict(X_Scaled_train), columns=['Bronzes Prediction'])
y_train['Bronzes Prediction'] = y_train['Bronzes Prediction']
y_train['Bronzes Prediction'] = y_train['Bronzes Prediction'].clip(lower=0)

y_train['Medals Prediction']  = y_train['Golds Prediction'] + y_train['Silvers Prediction'] + y_train['Bronzes Prediction']
y_train.head()

,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Year,NOC,Golds Prediction,Silvers Prediction,Bronzes Prediction,Medals Prediction
0,5,2,0,3,1900,AUS,1.638083,2.526214,3.560204,7.724501
1,6,0,3,3,1900,AUT,0.799500,1.223944,1.504581,3.528025
2,17,6,7,4,1900,BEL,1.347049,1.791576,1.513923,4.652548
3,2,0,1,1,1900,BOH,0.185417,0.684680,1.617472,2.487570
4,2,1,0,1,1900,CAN,0.133383,1.544888,2.150326,3.828597


In [55]:
# Intercepts
print(regr_golds.intercept_)
print(regr_silvers.intercept_)
print(regr_bronzes.intercept_)

4.194649446494465
4.15129151291513
4.542435424354244


In [56]:
# The coefficients
columns                          = X_train.columns.drop('NOC')
features                         = pd.DataFrame(columns.T, columns=['Feature'])
features['Golds Coefficients']   = regr_golds.coef_.T
features['Silvers Coefficients'] = regr_silvers.coef_.T
features['Bronzes Coefficients'] = regr_bronzes.coef_.T

features = features.sort_values(by='Golds Coefficients', ascending=False).reset_index().drop(['index'],axis=1)
features

,Feature,Golds Coefficients,Silvers Coefficients,Bronzes Coefficients
0,NOC_USA,2.343560,2.012559,1.680129
1,Host,1.938872,1.628620,1.283959
2,NOC_URS,1.788893,1.549796,1.386051
3,NOC_EUN,1.193785,1.001709,0.745854
4,NOC_GDR,1.054708,0.891282,0.728414
...,...,...,...,...
196,Sports_Lag9,-0.317118,-0.263364,-0.087779
197,Silver_Lag6,-0.327961,-0.414969,-0.242140
198,Athletes_Lag1,-0.364454,0.052335,0.045582
199,Bronze_Lag6,-0.380836,-0.335458,-0.504454


# Rio 2016 Validation Test Set

In [57]:
print(X_test.shape)
X_test.head()

(82, 202)


,Year,Host,Gold_Lag1,Silver_Lag1,Bronze_Lag1,Athletes_Lag1,Females_Lag1,Sports_Lag1,Events_Lag1,Athletes per Event_Lag1,...,NOC_UKR,NOC_URS,NOC_USA,NOC_UZB,NOC_VEN,NOC_VIE,NOC_YUG,NOC_ZAM,NOC_ZIM,NOC
0,2024,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000,...,False,False,False,False,False,False,False,False,False,ALB
1,2024,0,0.0,2.0,0.0,64.0,10.0,13.0,48.0,1.333,...,False,False,False,False,False,False,False,False,False,ALG
2,2024,0,0.0,1.0,2.0,189.0,58.0,31.0,61.0,3.098,...,False,False,False,False,False,False,False,False,False,ARG
3,2024,0,0.0,2.0,2.0,17.0,3.0,8.0,19.0,0.895,...,False,False,False,False,False,False,False,False,False,ARM
4,2024,0,17.0,7.0,22.0,489.0,261.0,41.0,193.0,2.534,...,False,False,False,False,False,False,False,False,False,AUS


In [58]:
# Make predictions using the test set
y_test['Golds Prediction']   = pd.DataFrame(
    regr_golds.predict(X_Scaled_test), columns=['Golds Prediction'])
y_test['Golds Prediction']   = y_test['Golds Prediction']
y_test['Golds Prediction']   = y_test['Golds Prediction'].clip(lower=0)

y_test['Silvers Prediction'] = pd.DataFrame(
    regr_silvers.predict(X_Scaled_test), columns=['Silvers Prediction'])
y_test['Silvers Prediction'] = y_test['Silvers Prediction']
y_test['Silvers Prediction'] = y_test['Silvers Prediction'].clip(lower=0)

y_test['Bronzes Prediction'] = pd.DataFrame(
    regr_bronzes.predict(X_Scaled_test), columns=['Bronzes Prediction'])
y_test['Bronzes Prediction'] = y_test['Bronzes Prediction']
y_test['Bronzes Prediction'] = y_test['Bronzes Prediction'].clip(lower=0)

y_test['Medals Prediction']  = y_test['Golds Prediction'] + y_test['Silvers Prediction'] + y_test['Bronzes Prediction']
y_test.head()

,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Year,NOC,Golds Prediction,Silvers Prediction,Bronzes Prediction,Medals Prediction
0,2,0,0,2,2024,ALB,1.988177,1.701571,1.940190,5.629938
1,3,2,0,1,2024,ALG,0.410786,0.503520,1.480044,2.394350
2,3,1,1,1,2024,ARG,1.707488,2.840809,3.047754,7.596050
3,4,0,3,1,2024,ARM,0.474217,1.147182,1.169518,2.790916
4,53,18,19,16,2024,AUS,13.915096,11.529488,15.291994,40.736578


In [ ]:
rio_2024_medals = medals[medals['Year'] == '2024']
rio_2024_medals = y_test[['NOC','Medals Prediction']].merge(rio_2024_medals, on='NOC')
rio_2024_medals = rio_2024_medals.sort_values(by='Medals Prediction', ascending=False)
rio_2024_medals.head()

,NOC,Medals Prediction,Year,Host,Athletes,Females,Sports,Events,Medals,Golds,Silvers,Bronzes,Real_Medals,Real_Gold,Real_Silver,Real_Bronze,Athletes per Event


In [ ]:
hover_text = []
color_range = []
for index, row in rio_2016_medals.iterrows():
    hover_text.append(('Games: {games}<br>'+
                       'Host: {host}<br>'+
                       'NOC: {noc}<br>'+
                       'Region: {region}<br>'+
                       'Athletes: {athletes}<br>'+
                       'Females: {females}<br>'+
                       'Sports: {sports}<br>'+
                       'Events: {events}<br>'+
                       'Athletes per Event: {ape}<br>'+
                       'Medals: {medals}<br>'+
                       'Medals Prediction: {medals_pred}<br>'+
                       'Golds: {golds}<br>'+
                       'Silvers: {silvers}<br>'+
                       'Bronzes: {bronzes}').format(games       = row['Games'],
                                                    host        = row['Host'],
                                                    noc         = row['NOC'],
                                                    region      = row['Region'],
                                                    athletes    = row['Athletes'],
                                                    females     = row['Females'],
                                                    sports      = row['Sports'],
                                                    events      = row['Events'],
                                                    ape         = row['Athletes per Event'],
                                                    medals      = row['Medals'],
                                                    medals_pred = row['Medals Prediction'],
                                                    golds       = row['Golds'],
                                                    silvers     = row['Silvers'],
                                                    bronzes     = row['Bronzes']))
    color_range.append(max(row['Events'],100))

trace0 = go.Scatter(
    x      = rio_2016_medals['Medals'],
    y      = rio_2016_medals['Medals Prediction'],
    text   = hover_text,
    mode   = 'markers',
    marker = dict(
        size          = rio_2016_medals['Athletes']/10,
        color         = color_range,
        showscale     = True,
        reversescale  = True,
        colorscale    = 'RdBu',
        colorbar      = dict(
                title = 'Total<br>Events'
            )
    )
)

data      = [trace0]
layout    = go.Layout(
    title = 'Medal Table of the Rio 2016 Summer Olympic Games',
    xaxis = dict(
        title         = 'Total Number of Actual Medals by Nation',
        gridcolor     = 'rgb(255, 255, 255)',
        zerolinewidth = 1,
        ticklen       = 5,
        gridwidth     = 2,
    ),
    yaxis = dict(
        title         = 'Total Number of Predicted Medals by Nation',
        gridcolor     = 'rgb(255, 255, 255)',
        zerolinewidth = 1,
        ticklen       = 5,
        gridwidth     = 2,
    ),
    paper_bgcolor = 'rgb(243, 243, 243)',
    plot_bgcolor  = 'rgb(243, 243, 243)',
)

fig = go.Figure(data = data, layout = layout)
py.iplot(fig, filename = 'rio-2016-olympic-medals')

KeyboardInterrupt: 

In [59]:
# Explained variance score: 1 is perfect prediction
print('Variance score: %.2f' % r2_score(y_train['Real_Medals'], y_train['Medals Prediction']))
print('Variance score: %.2f' % r2_score(y_test['Real_Medals'],  y_test['Medals Prediction']))

Variance score: 0.84
Variance score: 0.96


In [60]:
# The mean squared error
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_train['Real_Medals'], y_train['Medals Prediction'])))
print("Root mean squared error: %.2f" % np.sqrt(mean_squared_error(y_test['Real_Medals'],  y_test['Medals Prediction'])))

Root mean squared error: 8.67
Root mean squared error: 4.08


In [26]:
residuals = y_train['Medals Prediction'] - y_train['Real_Medals']
fig = sm.graphics.qqplot(residuals, dist=stats.norm, line='45', fit=True)
fig.show()

NameError: name 'sm' is not defined

# Los Angeles 2028 Prediction

In [27]:
previous_medals = medals[medals['Year'] >= 2016]
previous_medals = previous_medals.groupby(
    ['NOC'])[['Athletes','Females','Sports','Events']].mean()
previous_medals = previous_medals.reset_index()
print(previous_medals.shape)
previous_medals.sort_values(by='Athletes',ascending=False).head()

(104, 5)


,NOC,Athletes,Females,Sports,Events
99,USA,602.333333,319.666667,41.333333,241.333333
32,FRA,463.000000,212.000000,39.666667,206.333333
4,AUS,461.000000,245.666667,38.666667,197.333333
48,JPN,459.333333,218.666667,39.666667,191.333333
35,GER,430.000000,195.666667,36.000000,206.333333


In [28]:
X_test.sort_values(by='Athletes',ascending=False).head()

,Year,Host,Athletes,Females,Sports,Events,Athletes per Event,Gold_Lag1,Silver_Lag1,Bronze_Lag1,...,NOC_UKR,NOC_URS,NOC_USA,NOC_UZB,NOC_VEN,NOC_VIE,NOC_YUG,NOC_ZAM,NOC_ZIM,NOC
79,2024,0,619,328,47,234,2.645,39.0,41.0,33.0,...,False,False,True,False,False,False,False,False,False,USA
26,2024,601,601,295,46,223,2.695,10.0,12.0,11.0,...,False,False,False,False,False,False,False,False,False,FRA
4,2024,0,474,264,44,205,2.312,17.0,7.0,22.0,...,False,False,False,False,False,False,False,False,False,AUS
29,2024,0,457,225,42,196,2.332,10.0,11.0,16.0,...,False,False,False,False,False,False,False,False,False,GER
40,2024,0,430,201,41,166,2.590,27.0,14.0,17.0,...,False,False,False,False,False,False,False,False,False,JPN


In [99]:
los_2028_medals = medals.copy()  # 复制原始 DataFrame

# 获取所有 2024 年的数据
data_2024 = los_2028_medals[los_2028_medals['Year'] == 2024]

# 将 2024 年的数据复制到 2028 年（假设您希望 2028 年的数据与 2024 年一样）
data_2024['Year'] = 2028  # 将数据的 Year 列改为 2028

# 将 2024 年的数据添加到 DataFrame 中，作为 2028 年的数据
los_2028_medals = pd.concat([los_2028_medals, data_2024], ignore_index=True)


In [100]:
los_2028_medals=add_lag_features(los_2028_medals, lag=9)
los_2028_medals=los_2028_medals.drop(['Medals','Silvers','Bronzes','Golds','Athletes', 'Females', 'Sports', 'Events','Athletes per Event'],axis=1)
los_2028_medals.fillna(0,inplace=True)
NOC=los_2028_medals['NOC']
los_2028_medals        = pd.get_dummies(data=los_2028_medals, columns=['NOC'])
los_2028_medals['NOC'] = NOC
los_2028_medals=los_2028_medals.drop(['Real_Medals','Real_Gold','Real_Silver','Real_Bronze'],axis=1)
los_2028_medals=los_2028_medals[los_2028_medals['Year']==2028]
los_2028_medals

,Year,Host,Gold_Lag1,Silver_Lag1,Bronze_Lag1,Athletes_Lag1,Females_Lag1,Sports_Lag1,Events_Lag1,Athletes per Event_Lag1,...,NOC_UKR,NOC_URS,NOC_USA,NOC_UZB,NOC_VEN,NOC_VIE,NOC_YUG,NOC_ZAM,NOC_ZIM,NOC
1166,2028,0,0.0,0.0,2.0,8.0,3.0,4.0,9.0,0.889,...,False,False,False,False,False,False,False,False,False,ALB
1167,2028,0,2.0,0.0,1.0,46.0,19.0,15.0,47.0,0.979,...,False,False,False,False,False,False,False,False,False,ALG
1168,2028,0,1.0,1.0,1.0,143.0,33.0,25.0,41.0,3.488,...,False,False,False,False,False,False,False,False,False,ARG
1169,2028,0,0.0,3.0,1.0,15.0,2.0,7.0,16.0,0.938,...,False,False,False,False,False,False,False,False,False,ARM
1170,2028,0,18.0,19.0,16.0,474.0,264.0,44.0,205.0,2.312,...,False,False,False,False,False,False,False,False,False,AUS
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1243,2028,0,1.0,1.0,0.0,25.0,13.0,4.0,16.0,1.562,...,False,False,False,False,False,False,False,False,False,UGA
1244,2028,0,3.0,5.0,4.0,141.0,69.0,25.0,101.0,1.396,...,True,False,False,False,False,False,False,False,False,UKR
1245,2028,0,40.0,44.0,42.0,619.0,328.0,47.0,234.0,2.645,...,False,False,True,False,False,False,False,False,False,USA
1246,2028,0,8.0,2.0,3.0,88.0,31.0,17.0,68.0,1.294,...,False,False,False,True,False,False,False,False,False,UZB


In [101]:

# change host to Japan
los_2028_medals['Host']                                        = 0
los_2028_medals.loc[los_2028_medals['NOC'] == 'USA', 'Host'] = 1




#Reindex
los_2028_medals=los_2028_medals.reindex(list(X_train.columns),axis=1)


print(los_2028_medals.shape)
los_2028_medals.tail()

(82, 202)


,Year,Host,Gold_Lag1,Silver_Lag1,Bronze_Lag1,Athletes_Lag1,Females_Lag1,Sports_Lag1,Events_Lag1,Athletes per Event_Lag1,...,NOC_UKR,NOC_URS,NOC_USA,NOC_UZB,NOC_VEN,NOC_VIE,NOC_YUG,NOC_ZAM,NOC_ZIM,NOC
1243,2028,0,1.0,1.0,0.0,25.0,13.0,4.0,16.0,1.562,...,False,False,False,False,False,False,False,False,False,UGA
1244,2028,0,3.0,5.0,4.0,141.0,69.0,25.0,101.0,1.396,...,True,False,False,False,False,False,False,False,False,UKR
1245,2028,1,40.0,44.0,42.0,619.0,328.0,47.0,234.0,2.645,...,False,False,True,False,False,False,False,False,False,USA
1246,2028,0,8.0,2.0,3.0,88.0,31.0,17.0,68.0,1.294,...,False,False,False,True,False,False,False,False,False,UZB
1247,2028,0,0.0,0.0,1.0,31.0,22.0,5.0,8.0,3.875,...,False,False,False,False,False,False,False,True,False,ZAM


In [102]:
X_train.shape

(1084, 202)

In [103]:
los_2028_medals.describe()

,Year,Host,Gold_Lag1,Silver_Lag1,Bronze_Lag1,Athletes_Lag1,Females_Lag1,Sports_Lag1,Events_Lag1,Athletes per Event_Lag1,...,Events_Lag8,Athletes per Event_Lag8,Gold_Lag9,Silver_Lag9,Bronze_Lag9,Athletes_Lag9,Females_Lag9,Sports_Lag9,Events_Lag9,Athletes per Event_Lag9
count,82.0,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,...,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000,82.000000
mean,2028.0,0.012195,3.878049,3.841463,4.560976,118.341463,59.548780,18.073171,65.475610,1.533805,...,53.439024,0.918220,1.695122,1.926829,2.439024,72.756098,19.524390,8.292683,46.048780,0.844317
std,0.0,0.110432,7.308696,7.068395,6.815193,140.710299,75.262365,11.883987,58.687964,0.664035,...,66.214783,0.782118,4.617971,4.319867,4.422439,108.044751,35.955468,8.962870,60.044614,0.764796
min,2028.0,0.000000,0.000000,0.000000,0.000000,4.000000,1.000000,2.000000,4.000000,0.615000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,2028.0,0.000000,0.000000,0.000000,1.000000,26.000000,13.250000,9.000000,23.250000,1.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,2028.0,0.000000,1.000000,1.000000,2.000000,63.000000,29.500000,16.000000,47.000000,1.300500,...,27.500000,1.174000,0.000000,0.000000,0.000000,30.000000,1.000000,5.000000,22.500000,1.046500
75%,2028.0,0.000000,3.000000,3.000000,5.000000,141.000000,66.000000,24.750000,88.000000,1.924250,...,85.250000,1.559250,1.000000,2.000000,3.000000,95.000000,25.000000,16.000000,72.750000,1.522250
max,2028.0,1.000000,40.000000,44.000000,42.000000,619.000000,328.000000,47.000000,234.000000,3.875000,...,248.000000,2.177000,36.000000,31.000000,27.000000,526.000000,195.000000,27.000000,230.000000,2.287000


In [104]:
los_2028_medals_scaled = scaler.transform(los_2028_medals.drop('NOC',axis=1))
los_2028_medals_scaled


array([[ 1.44683107, -0.13520563, -0.46255216, ..., -0.10580184,
        -0.04299336, -0.03038686],
       [ 1.44683107, -0.13520563, -0.21390027, ..., -0.10580184,
        -0.04299336, -0.03038686],
       [ 1.44683107, -0.13520563, -0.33822622, ..., -0.10580184,
        -0.04299336, -0.03038686],
       ...,
       [ 1.44683107, -0.11969305,  4.51048572, ..., -0.10580184,
        -0.04299336, -0.03038686],
       [ 1.44683107, -0.13520563,  0.53205541, ..., -0.10580184,
        -0.04299336, -0.03038686],
       [ 1.44683107, -0.13520563, -0.46255216, ..., -0.10580184,
        23.2594067 , -0.03038686]])

In [105]:
# Make predictions using the Tokyo 2020 set
y_2028 = los_2028_medals[['Year','NOC']]
y_2028['Golds Prediction']   = pd.DataFrame(
    regr_golds.predict(los_2028_medals_scaled), columns=['Golds Prediction'])
Gold_lower_bound, Gold_upper_bound = bootstrap_resample(los_2028_medals_scaled, y_train['Real_Gold'])
y_2028['Gold_lower_bound']   = pd.DataFrame(
    Gold_lower_bound, columns=['Gold_lower_bound'])
y_2028['Gold_upper_bound']   = pd.DataFrame(
    Gold_upper_bound, columns=['Gold_upper_bound'])
y_2028['Golds Prediction']   = y_2028['Golds Prediction']
y_2028['Golds Prediction']   = y_2028['Golds Prediction'].clip(lower=0)


y_2028['Silvers Prediction'] = pd.DataFrame(
    regr_silvers.predict(los_2028_medals_scaled), columns=['Silvers Prediction'])
Silver_lower_bound, Silver_upper_bound = bootstrap_resample(los_2028_medals_scaled, y_train['Real_Silver'])
y_2028['Silver_lower_bound'] = pd.DataFrame(Silver_lower_bound, columns=['Silver_lower_bound'])
y_2028['Silver_upper_bound'] = pd.DataFrame(Silver_upper_bound, columns=['Silver_upper_bound'])
y_2028['Silvers Prediction'] = y_2028['Silvers Prediction']
y_2028['Silvers Prediction'] = y_2028['Silvers Prediction'].clip(lower=0)

y_2028['Bronzes Prediction'] = pd.DataFrame(
    regr_bronzes.predict(los_2028_medals_scaled), columns=['Bronzes Prediction'])
Bronze_lower_bound, Bronze_upper_bound = bootstrap_resample(los_2028_medals_scaled, y_train['Real_Bronze'])
y_2028['Bronze_lower_bound'] = pd.DataFrame(Bronze_lower_bound, columns=['Bronze_lower_bound'])
y_2028['Bronze_upper_bound'] = pd.DataFrame(Bronze_upper_bound, columns=['Bronze_upper_bound'])
y_2028['Bronzes Prediction'] = y_2028['Bronzes Prediction']
y_2028['Bronzes Prediction'] = y_2028['Bronzes Prediction'].clip(lower=0)

y_2028['Medals Prediction']  = y_2028['Golds Prediction'] + y_2028['Silvers Prediction'] + y_2028['Bronzes Prediction']
y_2028['Medals_lower_bound']  = y_2028['Gold_lower_bound'] + y_2028['Silver_lower_bound'] + y_2028['Bronze_lower_bound']
y_2028['Medals_upper_bound']  = y_2028['Gold_upper_bound'] + y_2028['Silver_upper_bound'] + y_2028['Bronze_upper_bound']

y_2028.head()

,Year,NOC,Golds Prediction,Gold_lower_bound,Gold_upper_bound,Silvers Prediction,Silver_lower_bound,Silver_upper_bound,Bronzes Prediction,Bronze_lower_bound,Bronze_upper_bound,Medals Prediction,Medals_lower_bound,Medals_upper_bound
1166,2028,ALB,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1167,2028,ALG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1168,2028,ARG,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1169,2028,ARM,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1170,2028,AUS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [90]:
los_2028_medals = y_2028.merge(los_2028_medals, on=['Year','NOC'])
los_2028_medals = los_2028_medals.sort_values(by='Medals Prediction', ascending=False).reset_index()
los_2028_medals[['NOC','Host','Gold_lower_bound',
                   'Golds Prediction','Gold_upper_bound','Silver_lower_bound','Silvers Prediction','Silver_upper_bound','Bronze_lower_bound','Bronzes Prediction','Bronze_upper_bound','Medals_lower_bound','Medals Prediction','Medals_upper_bound']].head(20)


,NOC,Host,Gold_lower_bound,Golds Prediction,Gold_upper_bound,Silver_lower_bound,Silvers Prediction,Silver_upper_bound,Bronze_lower_bound,Bronzes Prediction,Bronze_upper_bound,Medals_lower_bound,Medals Prediction,Medals_upper_bound
0,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
5,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
los_2028_medals.to_csv('los_2028_medals.csv', index=False)